# Exercise 1
Review the unstructured csv files and answer the following questions with code that supports your conclusions:

In [65]:
# Initial data ingestion

import sqlite3
import pandas as pd
import tabulate

conn = sqlite3.connect('fetch.sqlite')
df = pd.read_csv('USER_TAKEHOME.csv')
df.to_sql('users', conn, if_exists='replace', index=False)
df = pd.read_csv('PRODUCTS_TAKEHOME.csv')
df.to_sql('products', conn, if_exists='replace', index=False)
df = pd.read_csv('TRANSACTION_TAKEHOME.csv')
df.to_sql('transactions', conn, if_exists='replace', index=False)
conn.close

<function Connection.close()>

## 1. Users

### Are there any data quality issues present?

The largest data quality issue with the User data is not actually with the provided Users data; it is with the Transactions data. While there is a `user_id` for every row in Transactions, only a small number (91) of the `user_id`s exist in the `users` table. This will essentially prohibit transaction analysis from using user data and indicates that the user dataset is incomplete.

However, overall, the Users data that was provided is of good quality. It has a unique `id` key and every entry has a valid `created_date`. There is 96% coverage for `birth_date`, 95% for `state`, 69% for `language` and 94% for `gender`. I will rename `id` to `user_id` because that's my prefered style for analysis tables.

`created_date` is a proper timestamp terminating in a Z, which means it is in UTC.  I will rename `created_date` to `created_at` because it's a timestamp, not a date. 

`birth_date` is also stored as an UTC timestamp, but most of the values have no time component. However, some of the rows do have a time component. Many of those only have an hour component. My hypothesis is that this is an artifact of some database migration or an extract that converted timezones from local to UTC. If the original values were dates stored as timestamps, the conversation to UTC would have offset the values by some number of whole hours (or half hour for a few time zones.) However, there are some entries that have a minute and second component. I can only assume that was caused by some incorrect data collection process.  

`gender` has options that appear to be duplicative, for example 'prefer_not_to_say' vs. 'Prefer not to say'. Looking at the creation timestamps for these values, it appears that the options offered have been added and removed over time. Based on the timestamps and the values, some of these values can be combined. However, I will be very cautious in doing so in order to respect what the users selected.

The values 'male', 'female' and NULL all have entries created from 2014 through September 2024 (the maximum timestamp). The value 'transgender' first appears in 2017 and then stops in 2022. 

'prefer_not_to_say' first appears in 2015 and continues through the present. "Prefer not to say" only has one value on June of 2022. This may have been a bug where the display value was recorded rather than the standardized value. 

'non_binary' first appears in 2018 and continues through the present. 'Non-Binary' is only seen in May and June 2022, so that might be a similar or the same bug as above. 

'not_specified' only appears between December 2020 and January 2021. It appears to have been replaced by 'unknown', which only appears between January and August 2021. 'not_listed' first appears the following day in August 2021 and continues through the present, so it must have replaced 'unknown'. "My gender isn't listed" only appears between April and June of 2022, so it is likely the same bug as previously mentioned. 

Here's how I will modify the values stored in the gender column:
* female: keep
* male: keep
* NULL: keep
* transgender: keep
* prefer_not_to_say: keep
* Prefer not to say: merged into prefer_not_to_say
* non_binary: keep
* Non-Binary: merged into 'non_binary'
* not_listed: keep
* unknown: merged into not_listed
* not_specified: merged into 'not_listed'
* My gender isn't listed: merged into 'not_listed'

The other columns have no issues. `state` contains all 50 states plus PR and DC, which is as expected. `language` is mostly 'en', but with a few 'es-419'. These are IETF language tags and mean "English" and "Spanish appropriate for the Latin America and Caribbean region". 

### Are there any fields that are challenging to understand?
No, the fields are easy to understand and the minor data quality issues do not prevent that. 

In [ ]:
conn = sqlite3.connect('fetch.sqlite')
cur = conn.cursor()

# User data column coverage
cur.execute('''
select count(*) as n_rows
    , count(distinct id) as n_users
    , count(distinct case when created_date is not null then id else null end) as n_with_created_dates
    , count(distinct case when birth_date is not null then id else null end) as n_with_birth_dates
    , count(distinct case when state is not null then id else null end) as n_with_state
    , count(distinct case when language is not null then id else null end) as n_with_language
    , count(distinct case when gender is not null then id else null end) as n_with_gender
from users
''')

tabulate.tabulate(cur.fetchall(), tablefmt='html')

100000,100000,100000,96325,95188,69492,94108


In [ ]:
# state counts
cur.execute('''
select state, count(*) from users group by state order by count(*) desc
''')

tabulate.tabulate(cur.fetchall(), tablefmt='html')

TX,9028
FL,8921
CA,8589
NY,5703
,4812
IL,3794
GA,3718
PA,3644
OH,3519
NC,3399
NJ,2899


In [ ]:
# number of states
cur.execute('''
select count(distinct state) from users
''')

tabulate.tabulate(cur.fetchall(), tablefmt='html')

52


In [ ]:
# language counts
cur.execute('''
select language, count(*) from users group by language order by count(*) desc
''')

tabulate.tabulate(cur.fetchall(), tablefmt='html')

en,63403
,30508
es-419,6089


In [ ]:
# gender counts and first/last creation dates
cur.execute('''
select gender, count(*), min(created_date), max(created_date) from users group by gender order by count(*) desc
''')

tabulate.tabulate(cur.fetchall(), tablefmt='html')

female,64240,2014-07-09 01:24:05.000 Z,2024-09-11 17:58:04.000 Z
male,25829,2014-09-09 22:03:56.000 Z,2024-09-11 17:56:53.000 Z
,5892,2014-04-18 23:14:55.000 Z,2024-09-11 17:59:15.000 Z
transgender,1772,2017-03-05 19:37:20.000 Z,2022-08-06 21:07:23.000 Z
prefer_not_to_say,1350,2015-07-23 22:28:51.000 Z,2024-09-11 17:52:47.000 Z
non_binary,473,2018-11-28 23:02:36.000 Z,2024-09-11 17:53:02.000 Z
unknown,196,2021-01-12 19:03:07.000 Z,2021-08-27 03:47:51.000 Z
not_listed,180,2021-08-28 22:23:02.000 Z,2024-09-09 04:18:00.000 Z
Non-Binary,34,2022-05-15 22:17:26.000 Z,2022-06-19 13:44:09.000 Z
not_specified,28,2020-12-15 16:06:04.000 Z,2021-01-23 00:02:40.000 Z
My gender isn't listed,5,2022-04-02 00:47:40.000 Z,2022-06-20 22:56:27.000 Z


In [ ]:
# valid/invalid user counts
cur.execute('''
select case when u.id is not null then 'valid user' else 'invalid user' end as user_flag
, count(*) as n_rows
, count(distinct t.user_id) as n_users
from transactions as t
    left join users u on t.user_id = u.id
where t.user_id is not null
group by user_flag
''')

cur.close
conn.close

tabulate.tabulate(cur.fetchall(), tablefmt='html')


invalid user,49738,17603
valid user,262,91



## 2. Products

### Are there any data quality issues present?

The product data has several major data quality issues. First, and like the User data, it appears to be incomplete relative to the Transactions data. Only 56% of the Transactions with a value for `barcode` have a value that is listed in the Product data. This indicates that either there are a lot of low quality barcode entries in Transactions that are not validated against the Product data or that the Product data is incomplete. It also means that any analysis of Transactions data that joins it with Products will be giving an incomplete view. 

A small number (4,025 or 0.5%) of the product entries have a NULL value for `barcode`, which means they cannot be joined to Transactions. `barcode` is also not a unique identifier. There are 841,527 rows with a non-NULL barcode but only 841,342 unique barcodes. So it's a very small number of duplicate rows (185) but its enough to be a problem when joining the data. For my analysis, I will create a de-duplicated version of the Product data with no duplicates or NULL barcodes. 

Most of the duplicates are simple duplicates. However, I did see a few that had spelling differences or had "Placeholder Manufacturer" on one entry and the real manufacturer on the other. My de-duplication is pretty simple but a better version could be done that preferences real names over placeholder names. 

The rest of the columns seem fine. Category 1 & 2 data is largely complete with only 111 rows having no Category 1 value out of a total of 845,552 rows. For Category 2, 1,424 have no value. Category 3 is less complete with 60,566 rows with no value (7% of the total). Category 4 is almost never used with only 67,459 row (8%) having a value. 

In addition, 99% of the rows are in the top two Category 1 values: Health & Wellness and Snacks. This reinforces my suspicion that the Product data is incomplete. 

Manufacturer is on 73% of rows. The most common Manufacturer is 'Placeholder Manufacturer' which makes up 14% of Manufacturer values. Brand is also on 73% of rows. The most common Brand is 'REM BRAND' which makes up 3% of Manufacturer values. 'Brand Not Known' is the second most common brand and also makes up 3%.

### Are there any fields that are challenging to understand?

No, the fields make sense. My only confusion is why there are duplicate and missing barcodes and why Category 4 is blank on so many entries. 

In [18]:
conn = sqlite3.connect('fetch.sqlite')
cur = conn.cursor()

In [ ]:
# valid/invalid barcodes
cur.execute('''
select case when p.barcode is not null then 'barcode in products' else 'barcode not in products' end as barcode_flag
, count(*) as n_rows
from transactions as t
    left join products p on t.barcode = p.barcode
where t.barcode is not null
group by barcode_flag
''')
tabulate.tabulate(cur.fetchall(), tablefmt='html')


barcode in products,24854
barcode not in products,19408


In [ ]:
# count of rows with/without barcodes
cur.execute('''
select case when barcode is not null then 'barcode' else 'no barcode' end as barcode_flag
, count(*) as n_rows
from products
group by barcode_flag
''')
tabulate.tabulate(cur.fetchall(), tablefmt='html')

barcode,841527
no barcode,4025


In [66]:
# rows with duplicate barcodes
cur.execute('''
with products_rn as 
        (
        select barcode, manufacturer, brand, category_1, category_2, category_3, category_4
        , count() over (partition by barcode) as dupe_count
        from products
        where barcode is not null
        )
    select barcode, manufacturer, brand, category_1, category_2, category_3, category_4
    from products_rn
    where dupe_count > 1
    limit 20
''')
tabulate.tabulate(cur.fetchall(), tablefmt='html')

400510,MARS WRIGLEY,STARBURST,Snacks,Candy,Confection Candy,
400510,MARS WRIGLEY,STARBURST,Snacks,Candy,Confection Candy,
404310,PLACEHOLDER MANUFACTURER,BRAND NOT KNOWN,Snacks,Candy,Chocolate Candy,
404310,MARS WRIGLEY,M&M'S,Snacks,Candy,Chocolate Candy,
438711,TRADER JOE'S,TRADER JOE'S,Snacks,Crackers,Graham Crackers,
438711,TRADER JOE'S,TRADER JOE'S,Snacks,Crackers,Graham Crackers,
563178,TRADER JOE'S,TRADER JOE'S,Snacks,Fruit & Vegetable Snacks,Dried Vegetables,
563178,TRADER JOE'S,TRADER JOE'S,Snacks,Fruit & Vegetable Snacks,Dried Vegetables,
603898,TRADER JOE'S,TRADER JOE'S,Snacks,Nuts & Seeds,Cashews,
603898,TRADER JOE'S,TRADER JOE'S,Snacks,Nuts & Seeds,Cashews,
701983,SUNRIDGE FARMS,SUNRIDGE FARMS,Snacks,Nuts & Seeds,Snack Seeds,


In [ ]:
# column data coverage
cur.execute('''
select count(*) as n_rows
    , count(distinct barcode) as n_barcodes
    , sum(case when barcode is not null then 1 else 0 end) n_with_barcodes
    , sum(case when category_1 is not null then 1 else 0 end) n_with_category_1
    , sum(case when category_2 is not null then 1 else 0 end) n_with_category_2
    , sum(case when category_3 is not null then 1 else 0 end) n_with_category_3
    , sum(case when category_4 is not null then 1 else 0 end) n_with_category_4
    , sum(case when brand is not null then 1 else 0 end) n_with_brand
    , sum(case when manufacturer is not null then 1 else 0 end) n_with_manufacturer
from products
''')
tabulate.tabulate(cur.fetchall(), tablefmt='html')

845552,841342,841527,845441,844128,784986,67459,619080,619078


In [ ]:
# top category
cur.execute('''
select category_1
    , count(*)
from products
    where category_1 is not null
group by category_1
order by count(*) desc
limit 10
''')
tabulate.tabulate(cur.fetchall(), tablefmt='html')

Health & Wellness,512695
Snacks,324817
Beverages,3990
Pantry,871
Apparel & Accessories,846
Dairy,602
Needs Review,547
Alcohol,503
Home & Garden,115
Restaurant,69


In [ ]:
# top brands
cur.execute('''
select brand
    , count(*)
from products
    where brand is not null
group by brand
order by count(*) desc
limit 10
''')
tabulate.tabulate(cur.fetchall(), tablefmt='html')

REM BRAND,20813
BRAND NOT KNOWN,17025
PRIVATE LABEL,13467
CVS,6400
SEGO,4831
MEIJER,4050
DOVE,3857
RITE AID,3238
MATRIX,2958
EQUATE,2946


In [ ]:
# top manufacturers
cur.execute('''
select manufacturer
    , count(*)
from products
    where manufacturer is not null
group by manufacturer
order by count(*) desc
limit 10
''')
tabulate.tabulate(cur.fetchall(), tablefmt='html')

PLACEHOLDER MANUFACTURER,86902
PROCTER & GAMBLE,21065
REM MANUFACTURER,20813
UNILEVER,16864
L'OREAL,16699
PEPSICO,14468
JOHNSON & JOHNSON,10443
THE HERSHEY COMPANY,10222
MARS WRIGLEY,9732
MONDELĒZ INTERNATIONAL,7250


In [30]:
cur.close
conn.close

<function Connection.close()>

## 3. Transactions

### Are there any data quality issues present?

As previously mentioned, the Transactions data contains values for `barcode` that are not seen in the Product data set and values for `user_id` that are not seen in the Users data.

There is no unique key for the Transactions data. There are 50,000 rows but only 24,440 unique values for `receipt_id` which means that the average `receipt_id` appears 2.04 times. There is also no line item ID to go along with the receipt ID. 

Only 88% of rows have a value for `barcode`, which further limits the ability to combine Transactions data with Product data for analysis. 

However, there is 100% data coverage for the other columns: `purchase_date`, `scan_date`, `store` and `user_id`. 

`scan_date` is actually a timestamp, so I will rename it to `scan_at`. 

Transactions are all scanned between June 12th and September 8th of 2024. Purchase dates are all in the same date range. The purchase date is often the same day as the scan, but not always. Scan is usually after purchase, but not always. I'd assume that the purchase date is manually entered by the customer and the scan date is machine generated (ie when a receipt is scanned in).

Walmart is by far the most common store, with 40% of the entries. However, there is a very long tail. 

### Are there any fields that are challenging to understand?

Yes. `receipt_id` appears to be a "basket" ID but I hesitate to assume that since there's no matching ID for line items. 

The columns `final_quantity` and `final_sale` and how they interact is very confusing. `final_quantity` is a float value between 0.01 and 276, unless it has the string value of 'zero'. If `final_quantity` is 'zero', then there is always a `final_sale` value which is a float between 0 and a maximum value of 462.82. However, if `final_quantity`has a float value, `final_sale` can still have it's own float value. But it can also be empty with a value of ' ' (not NULL or ''). 

Of the 50K total rows, 37,500 have a valid non-zero value for `final_quantity` and 12,500 have 'zero'. Of the 37,500 with a valid quantity, 12,500 have a blank sale and the remaining 25,000 have a valid sale. So half of the rows have valid values for both quantity and sale, a quarter are missing one and a quarter are missing the other. This indicates to me that there is something artificial or constructed about the values I'm seeing for `final_quantity` and `final_sale`. 

Due to these issues, I will replace the 'zero' and ' ' values with NULL. 

In [ ]:
conn = sqlite3.connect('fetch.sqlite')
cur = conn.cursor()

In [ ]:
# rows with a barcode
cur.execute('''
select count(*) as n_rows
from transactions as t
where t.barcode is not null
''')

tabulate.tabulate(cur.fetchall(), tablefmt='html')

44238


In [ ]:
# column summary
cur.execute('''
select count(*) as n_rows
    , count(distinct receipt_id) as n_receipts
    , count(case when purchase_date is not null then receipt_id else null end) as n_with_purchase_dates
    , count(case when scan_date is not null then receipt_id else null end) as n_with_scan_dates
    , count(case when store_name is not null then receipt_id else null end) as n_with_store_name
    , count(case when user_id is not null then receipt_id else null end) as n_with_user_id
    , count(case when barcode is not null then receipt_id else null end) as n_with_barcode
from transactions
''')
tabulate.tabulate(cur.fetchall(), tablefmt='html')

50000,24440,50000,50000,50000,50000,44238


In [34]:
# look at time period for data
cur.execute('''
select min(scan_date), max(scan_date), min(purchase_date), max(purchase_date)
from transactions
''')
tabulate.tabulate(cur.fetchall(), tablefmt='html')

2024-06-12 06:36:34.910 Z,2024-09-08 23:07:19.836 Z,2024-06-12,2024-09-08


In [ ]:
# scans before purchase date
cur.execute('''
select count(*)
from transactions
where date(scan_date) < purchase_date
''')
tabulate.tabulate(cur.fetchall(), tablefmt='html')

94


In [ ]:
# min/max values of final sale
cur.execute('''
select min(cast(final_sale as decimal)), max(cast(final_sale as decimal))
from transactions
where final_sale <> ' '
''')

tabulate.tabulate(cur.fetchall(), tablefmt='html')

0,462.82


In [42]:
# min/max values of final quantity
cur.execute('''
select min(cast(final_quantity as decimal)), max(cast(final_quantity as decimal))
from transactions
where final_quantity <> 'zero'
''')

tabulate.tabulate(cur.fetchall(), tablefmt='html')

0.01,276


In [67]:
#final_quantity values when final_sale = ' '

cur.execute('''
select final_quantity, count(*) as n_rows
from transactions
where final_sale = ' '
group by final_quantity order by final_sale asc
limit 20
''')

tabulate.tabulate(cur.fetchall(), tablefmt='html')

''

In [68]:
#final_sale values when final_quantity = 'zero'

cur.execute('''
select final_sale, count(*) as n_rows
from transactions
where final_quantity = 'zero'
group by final_sale order by final_sale asc
limit 20
''')

tabulate.tabulate(cur.fetchall(), tablefmt='html')


''

In [51]:
# frequency of blank quantity
cur.execute('''
select case when final_quantity = 'zero' then 'zero quantity' else 'valid quantity' end as zero_flag
, count(*) as n_rows
from transactions
group by zero_flag
''')

tabulate.tabulate(cur.fetchall(), tablefmt='html')

valid quantity,37500
zero quantity,12500


In [50]:

cur.execute('''
select 
    case 
        when final_sale = ' ' and final_quantity = 'zero' then 'blank quantity and blank sale'  -- shouldn't ever exist
        when final_sale = ' ' and final_quantity <> 'zero' then 'blank sale, valid quantity' 
        when final_sale <> ' ' and final_quantity = 'zero' then 'valid sale and blank quantity' 
        when final_sale <> ' ' and final_quantity <> 'zero' then 'valid sale and valid quantity' 
        else 'not blank sale' end as sale_flag
, count(*) as n_rows
from transactions
group by sale_flag
order by n_rows desc
''')

tabulate.tabulate(cur.fetchall(), tablefmt='html')

valid sale and valid quantity,25000
valid sale and blank quantity,12500
"blank sale, valid quantity",12500


In [52]:
cur.close
conn.close

<function Connection.close()>

## Data Cleanup & Transformation Steps

In [64]:
conn = sqlite3.connect('fetch.sqlite')
cur = conn.cursor()
# Products
# Create a clean table

cur.execute('''
create table products_clean as 
    with products_rn as 
        (
        select barcode, manufacturer, brand, category_1, category_2, category_3, category_4
        , row_number() over (partition by barcode order by category_1 nulls last, category_2 nulls last, category_3 nulls last) as barcode_rn
        from products
        where barcode is not null
        )
    select barcode, manufacturer, brand, category_1, category_2, category_3, category_4
    from products_rn
    where barcode_rn = 1;
''')

# Users

# rename created_date to created_at because it's a timestamp, not a date
cur.execute('''
alter table users rename column created_date to created_at;
''')

# rename `id` to `user_id` because that's my prefered style for analysis tables
cur.execute('''
alter table users rename column id to user_id;
''')

# map values for gender
cur.execute('''
update users set gender = 'not_listed' where gender='unknown';
''')
cur.execute('''
update users set gender = 'not_listed' where gender='not_specified';
''')
cur.execute('''
update users set gender = 'not_listed' where gender='My gender isn''t listed';
''')
cur.execute('''
update users set gender = 'non_binary' where gender='Non-Binary';
''')
cur.execute('''
update users set gender = 'prefer_not_to_say' where gender='Prefer not to say';
''')

# Transactions 

# rename scan_date to scan_at because it's a timestamp, not a date
cur.execute('''
alter table transactions rename column scan_date to scan_at;
''')

# replace final_quantity = 'zero' with NULL
cur.execute('''
update transactions set final_quantity = NULL where final_quantity = 'zero';
''')

# replace final_sale = ' ' with NULL
cur.execute('''
update transactions set final_sale = NULL where final_sale = ' ';
''')

cur.close
conn.close

<function Connection.close()>